In [49]:
import pandas as pd, docx2txt
from io import StringIO
import os
import re

In [50]:
#read .docx into jupyter
text = docx2txt.process ("C://Users/Bryant/Documents/GitHub/ACT/archive/DataPrep/ACT Math Sheets_v2_3_9_18/Mean, Median, Mode.docx")

In [51]:
text;

## Clean text. Each question will need the following categories:

#### - ID (to be used as a Primary Key in MySQL db)
#### - Test Date
#### - Question Number
#### - Question Statement
#### - Answer Choices (one choice for each column, 5 total)
#### - Picture (if necessary)

In [52]:
#split text into list separated by \n\n i.e. 'enter' button
text_split = [line for line in StringIO(text).read().split('\n\n')]

In [53]:
#remove empty entries and spaces
try:
    text_split.remove('')
    text_split.remove(' ')
except:
    pass

In [54]:
text_split;

In [55]:


#store topic
topic = text_split[0]
full_list = []
new = []
for line in text_split[1:]:
    #store month and year of test date
    if re.search('(^December|June|April|July)', line):
        month = re.search('(^December|June|April|July)', line).group()
        year = re.search('$2010|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020',line).group()
        #continue to next iteration if test date is found
        continue
        
    #look for question number to initiate a new row entry
    #len(line) > 10 to avoid finding answers that begin with numbers
    if re.search('^..\.', line) and len(line) > 10:
        #append existing list to full_list, then reinitiate 'new' row
        if len(new) > 0:
            full_list.append(new)
            new = []
        #append various data for 'new' row entry 
        num = re.search('^..\.', line).group().replace(".","")
        #id column
        new.append(month[0:3] + "." + year + "." + num)
        new.append(topic)
        new.append(month + "." + year)
        new.append(num)
        #question statement
        new.append(line)
        continue

    #same as previous block with exception re.search looking for single digits
    if re.search('^.\.', line) and len(line) > 10:
        if len(new) > 0:
            full_list.append(new)
            new = []
        #store question  number    
        num = re.search('^.\.', line).group().replace(".","")
        new.append(month[0:3] + "." + year + "." + num)
        new.append(topic)
        new.append(month + "." + year)
        new.append(num)
        new.append(line)
        continue
    #only lines that reach 'else' statement should be answer choices; append to list   
    else:
        new.append(line)

In [56]:
full_list;

## ready file for import into MySQL db

In [57]:
#prep for import into MySQL db

from pandas import DataFrame
#convert list to df
df = DataFrame.from_records(full_list)
#export to .csv excluding index and header
df.to_csv('MeanMedianMode.csv', index=False, header=False)